In [ ]:
!pip install -U ultralytics opencv-python onnx onnxruntime


In [ ]:
from ultralytics import YOLO
import numpy as np, cv2, os
from google.colab import files
from google.colab.patches import cv2_imshow

In [ ]:
def detect_and_draw(img_bgr, conf=0.25, iou=0.45, imgsz=640, device="cpu",  preview_max_side=720):

    res = model(img_bgr, imgsz=imgsz, conf=conf, iou=iou, device=device)[0]
    vis = res.plot()  # малює бокси/лейбли на копії зображення (BGR)

    #маштабуемо вихідне зображення
    if preview_max_side is not None:
        h, w = vis.shape[:2]
        s = max(h, w)
        if s > preview_max_side:
            scale = preview_max_side / s
            vis = cv2.resize(vis, (int(w*scale), int(h*scale)), interpolation=cv2.INTER_AREA)

    #для логу/збереження
    dets = [
        {
          "cls_id": int(c),
          "cls_name": res.names[int(c)],
          "conf": float(p),
          "xyxy": [int(x) for x in b]
        }
        for b, c, p in zip(res.boxes.xyxy.cpu().numpy(),
                           res.boxes.cls.cpu().numpy(),
                           res.boxes.conf.cpu().numpy())
    ]
    return vis, dets
model = YOLO('yolov8m.pt')
model.export(format='onnx', opset=12)

In [ ]:
# відкриє кнопку "Choose files"
uploaded = files.upload()

for fname, content in uploaded.items():
    # декодуємо будь-який поширений формат у BGR (без проміжного JPG)
    arr = np.frombuffer(content, np.uint8)
    img = cv2.imdecode(arr, cv2.IMREAD_COLOR)
    if img is None:
        print(f"[!] File {fname} not found ")
    else:
      print('Image decode ✅')

vis, dets = detect_and_draw(img, preview_max_side=600)                 # запускаємо детекцію й малювання
cv2_imshow(vis)